<a href="https://colab.research.google.com/github/JessicaArauj/E2E_Automation_with_N8N_AWS_Python_and_PySpark/blob/main/extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install boto3 -q --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.2 MB/s eta 0:00:00


# Imports

In [ ]:
import io
import os
import zipfile

import boto3
import requests
from pyspark.sql import SparkSession
from tqdm import tqdm

from google.colab import userdata

# Variables

In [ ]:
AWS_ACCESS_KEY_ID = userdata.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = userdata.get('AWS_SECRET_ACCESS_KEY')
AWS_REGION = userdata.get('AWS_REGION')
BUCKET_NAME = userdata.get('BUCKET_NAME')

# Download and upload to S3 on AWS
Data from fixed broadband access available on the website https://anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip

In [ ]:
spark = SparkSession.builder.appName("S3DataLake").getOrCreate()

s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION,
)

download_url = "https://anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip"
response = requests.get(download_url, stream=True)
response.raise_for_status()
zip_bytes = io.BytesIO(response.content)

zip_key = "file/acessos_banda_larga_fixa.zip"
try:
    s3.upload_fileobj(zip_bytes, BUCKET_NAME, zip_key)
    print(f"ZIP uploaded completed to S3: {BUCKET_NAME}/{zip_key}")
except Exception as e:
    print("Error uploading to S3:", e)

objects = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix="file/")
for obj in objects.get("Contents", []):
    print("S3 Object:", obj["Key"])